In [21]:
import pandas as pd
import seaborn as sns
import sklearn
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import os
from pathlib import Path

In [47]:
year = 2019
for root, dirs, files in os.walk(f'../../../data/ncaa/raw/{year}/team_game_by_game/'):
    for f in files:
        df = pd.read_csv(Path(root).joinpath(f), header=1)
        df.drop(columns=["MP", "Attend", "BHE", "Unnamed: 20"], inplace=True)
        df.replace({'/':''}, regex=True, inplace=True)
        df.fillna(0, inplace=True)
        df[["Kills", "Errors", "Total Attacks", "Assists", "Aces", "SErr", "Digs", "RErr", "Block Solos", "Block Assists", "BErr"]] = df[["Kills", "Errors", "Total Attacks", "Assists", "Aces", "SErr", "Digs", "RErr", "Block Solos", "Block Assists", "BErr"]].astype(int)
        outpath = Path(root).parent.parent.parent.joinpath(f"processed/{year}/game_by_game_cleaned/")
        outpath.mkdir(parents=True, exist_ok=True)
        df.to_csv(outpath.joinpath(f), index=False)

In [48]:
year = 2019
dfs = []
team_names = []
for root, dirs, files in os.walk(f'../../../data/ncaa/processed/{year}/game_by_game_cleaned'):
    for f in files:
        team_names.append(f[:-4])
        dfs.append(pd.read_csv(Path(root).joinpath(f)))
print(year, len(dfs))

2019 332


In [49]:
dfs[130]

,Date,Opponent,Result,S,Kills,Errors,Total Attacks,Hit Pct,Assists,Aces,SErr,Digs,RErr,Block Solos,Block Assists,BErr,PTS
0,8302019,Cal Poly @ Salt Lake City,W 3 - 0,3,44,10,105,0.324,42,5,5,52,4,1,16,0,58.0
1,8302019,@ Utah,L 0 - 3,3,36,30,107,0.056,35,3,5,36,3,0,10,0,44.0
2,8312019,Saint Mary's (CA) @ Salt Lake City,W 3 - 0,3,42,11,86,0.360,39,6,6,30,3,2,22,2,61.0
3,9062019,"Creighton @ Cedar Falls, IA",L 1 - 3,4,55,22,152,0.217,49,3,3,67,4,0,8,4,62.0
4,9062019,"Southern California @ Cedar Falls, IA",W 3 - 1,4,63,25,176,0.216,61,6,10,76,4,2,8,3,75.0
5,9072019,@ UNI,W 3 - 0,3,40,14,99,0.263,38,6,8,48,1,2,14,3,55.0
6,9132019,Cleveland St.,W 3 - 0,3,45,9,103,0.350,44,6,5,48,2,1,8,1,56.0
7,9132019,FGCU,W 3 - 1,4,69,26,166,0.259,63,3,11,71,5,3,14,2,82.0
8,9142019,Indiana,L 2 - 3,5,57,29,161,0.174,56,3,6,72,6,0,27,2,73.5
9,9182019,Purdue,L 2 - 3,5,66,32,189,0.180,56,4,5,73,3,3,14,2,80.0
